In [ ]:
"""
This file is part of lc-power-match-baluns.
Copyright © 2023 Technical University of Denmark (developed by Rasmus Jepsen)

This library is free software; you can redistribute it and/or
modify it under the terms of the GNU Lesser General Public
License as published by the Free Software Foundation; either
version 2.1 of the License, or (at your option) any later version.

This library is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public
License along with this library; if not, write to the Free Software
Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA  02110-1301  USA
"""

In [ ]:
"""
This notebook shows that the Extended Pi topology shown below
cannot be a power matching balun for arbitrary complex impedances.
"""

In [ ]:
# import modules
from lcapy import Circuit, oo, expr, symbol, j, Eq, Z, Matrix, limit
from lcapy.expr import symbols

In [ ]:
# create lcapy circuit from netlist
# the balanced port is between nodes 1 and 3, and the unbalanced port is between nodes 2 and 0
balun_cct = Circuit("""
Z1 1 3; down
Z2 1 5; right
Z3 5 4; down
W 4 0; right
W 3 4; right
Z4 5 2; right
""")
balun_cct.draw()

In [ ]:
# create a differential mode two-port model
balun_twoport = balun_cct.twoport(1, 3, 2, 0)

In [ ]:
# retrieve the three-port Z parameters
balun_threeport_z = balun_cct.Zparamsn(2,0,1,0,3,0)
balun_threeport_z

In [ ]:
# initialise symbols

# resistances of the balanced and unbalanced ports
r_b, r_u = symbols('R_B R_U', real=True)

# reactances of the balanced and unbalanced ports
x_b, x_u = symbols('X_B X_U', real=True)

# impedances of the balanced and unbalanced ports
z_b, z_u = symbols('Z_B Z_U', complex=True)

# element reactances
x_1, x_2, x_3, x_4 = symbols('X_1 X_2 X_3 X_4', real=True)

In [ ]:
# The technique described in [1] is used to convert the three-port impedance matrix to three-port scattering parameters.

In [ ]:
# initialise matrices for converting Z-parameters to S-parameters

f = Matrix(((z_u.real ** 0.5 / 2, 0, 0), (0, (z_b.real / 2) ** 0.5 / 2, 0), (0, 0, (z_b.real / 2) ** 0.5 / 2)))

g = Matrix(((z_u, 0, 0), (0, z_b / 2, 0), (0, 0, z_b / 2)))

g_plus = Matrix(((z_u.conj, 0, 0), (0, z_b.conj / 2, 0), (0, 0, z_b.conj / 2)))

In [ ]:
# calculate the renormalised three-port scattering parameters
balun_threeport_s = f * (balun_threeport_z - g_plus) * (balun_threeport_z + g).inv() * f.inv()
balun_threeport_s

In [ ]:
# The method described in [2] is used to calculate the common-mode rejection ratio (CMRR).

In [ ]:
# calculate common-mode response
balun_cct_s21cs = expr('1/sqrt(2)') * (balun_threeport_s[1,0] + balun_threeport_s[2,0])
balun_cct_s21cs.simplify()

In [ ]:
# calculate differential-mode response
balun_cct_s21ds = expr('1/sqrt(2)') * (balun_threeport_s[1,0] - balun_threeport_s[2,0])
balun_cct_s21ds.simplify()

In [ ]:
# variable substitutions for later steps
substitutions = {'Z1': j * x_1, 'Z2': j * x_2, 'Z3': j * x_3, 'Z4': j * x_4, 'Z_B': r_b + j * x_b, 'Z_U': r_u + j * x_u}

In [ ]:
# find the inverse of the CMRR (this should be 0 for an ideal balun)
inv_cmrr = (balun_cct_s21cs / balun_cct_s21ds).subs(substitutions).simplify()
inv_cmrr

In [ ]:
"""Because the elements cannot be configured to yield an infinite CMRR,
this topology cannot be used as a balun.
"""

In [ ]:
"""References:
[1] K. Kurokawa, "Power waves and the scattering matrix," IEEE Transactions on Microwave Theory and Techniques, vol. 13, no. 2, pp. 194–202, 1965.
[2] D. Bockelman and W. Eisenstadt, "Combined differential and common-mode analysis of power splitters and combiners," IEEE Transactions on Microwave Theory and Techniques, vol. 43, no. 11, pp. 2627–2632, 1995.
"""